## Basic LLM Tasks
In the [*1_basic_prompt*](./1_basic_prompt.ipynb) notebook, we explored  prompts to query an LLM model. This notebook goes a bit futher with diverse examples to demonstrate various tasks, emphasizing effective usage of prompt engineering through practical instances. The tasks explored in this notebook, using sophiscated prompting techniques, show *how-to* code examples for common natural language understanfing capabilites of a generalized LLM, such as ChatGPT and Llama 2 series:

 * Text generation or completion
 * Text summarization
 * Text extraction
 * Text classification or sentiment analysis
 * Text categorization
 * Text transformation and translation
 * Simple and complex reasoning

<img src="./images/prompt_req_resp.png" height="35%" width="%65">


In [1]:
import warnings
import os

import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv

In [2]:
_ = load_dotenv(find_dotenv()) # read local .env file
warnings.filterwarnings('ignore')
openai.api_base = os.getenv("ANYSCALE_API_BASE", os.getenv("OPENAI_API_BASE"))
openai.api_key = os.getenv("ANYSCALE_API_KEY", os.getenv("OPENAI_API_KEY"))
MODEL = os.getenv("MODEL")
print(f"Using MODEL={MODEL}; base={openai.api_base}")

Using MODEL=meta-llama/Llama-2-70b-chat-hf; base=https://console.endpoints.anyscale.com/m/v1


In [3]:
from openai import OpenAI

client = OpenAI(
    api_key = openai.api_key,
    base_url = openai.api_base
)

In [4]:
def get_commpletion(clnt: object, model: str, system_content: str, user_content:str) -> str:
    chat_completion = clnt.chat.completions.create(
        model=model,
    messages=[{"role": "system", "content": system_content},
              {"role": "user", "content": user_content}],
    temperature = 0.8)

    response = chat_completion.choices[0].message.content
    return response

## Text generation or completion
In this simple task, we use an LLM to generate text by finishing an incomplete user content provided in the prompt. For example,
by providing an incomplete prompt such as "On a cold winter night, the stray dog ...". 

Let's try a few text generation or completion tasks by providing partial prompts in the user content. You will surprised at its
fluency and coherency in the generated text.

In [5]:
system_content = """You are master of all knowledge, and a helpful sage.
                    You must complete any incomplete sentence by drawing from your vast
                    knowledge about history, literature, science, social science, philosophy, religion, economics, sports, etc.
                  """

user_prompts =  ["On cold winter nights, the wolves in Siberia ...",
                 "On the day Franklin Benjamin realized his passion for printer, ...",
                 "During the final World Cup 1998 when France beat Brazil in Paris, ...",
                 "Issac Newton set under a tree when an apple fell..."
                ]

In [6]:
print(f"Using Endpoints: {openai.api_base} ...\n")
for user_prompt in user_prompts:
    prompt = f""" Compete the text ```{user_prompt}``` between three backticks.
    You will use simple, compound, and compound-complex sentences for all your responses, 
    and no more than one paragraph and no more than five sentences. 
    Keep your sentences succinct and cohesive.
    """
    response = get_commpletion(client, MODEL, system_content, prompt)
    response = response.replace("```", "")
    print(f"\nPrompt: {user_prompt}")
    print(f"\nAnswer: {response}")

Using Endpoints: https://console.endpoints.anyscale.com/m/v1 ...


Prompt: On cold winter nights, the wolves in Siberia ...

Answer:   Sure, I'd be happy to help! Here's a possible completion of the sentence:

On cold winter nights, the wolves in Siberia howl with a haunting melancholy, their piercing cries echoing through the frozen tundra as they roam in search of prey. Their thick fur coats and sharp instincts help them survive in this unforgiving environment, where temperatures can drop to -40 degrees Celsius and the snowdrifts stretch for miles. Despite their fearsome reputation, these majestic creatures are also surprisingly social, often forming close-knit packs that work together to hunt and protect their territory. In fact, scientists have observed that wolves in Siberia have a complex communication system, using a variety of vocalizations, body language, and even scent markings to communicate with each other. By studying their behavior, researchers have gained valuable insigh

## Text summarization

A common task in natural langauge processing is text summiarization. A common use case
is summarizing large articles or documents, for a quick and easy-to-absorb summaries.

You can instruct LLM to generate the response in a preferable style, and comprehensibility. For example, use simple language aimed for a certain grade level, keep the orginal style of the article, use different sentence sytles (as we have done in few of examples in this notebook and previous one).

Let's try a few examples.

In [7]:
system_content = """You are master of all knowledge about history, literature, science, philosophy, religion, economics, sports, etc.""" 
                
user_prompts = [
    """ The emergence of large language models (LLMs) has marked a significant 
         breakthrough in natural language processing (NLP), leading to remarkable 
         advancements in text understanding and generation. 
         
         Nevertheless, alongside these strides, LLMs exhibit a critical tendency 
         to produce hallucinations, resulting in content that is inconsistent with 
         real-world facts or user inputs. This phenomenon poses substantial challenges 
         to their practical deployment and raises concerns over the reliability of LLMs 
         in real-world scenarios, which attracts increasing attention to detect and 
         mitigate these hallucinations. In this survey, we aim to provide a thorough and 
         in-depth  overview of recent advances in the field of LLM hallucinations. 
         
         We begin with an innovative taxonomy of LLM hallucinations, then delve into the 
         factors contributing to hallucinations. Subsequently, we present a comprehensive
         overview of hallucination detection methods and benchmarks. 
         Additionally, representative approaches designed to mitigate hallucinations 
         are introduced accordingly. 
         
         Finally, we analyze the challenges that highlight the current limitations and 
         formulate open questions, aiming to delineate pathways for future  research on 
         hallucinations in LLMs.""",
    """  Can a Large Language Model (LLM) solve simple abstract reasoning problems?
         We explore this broad question through a systematic analysis of GPT on the 
         Abstraction and Reasoning Corpus (ARC), a representative benchmark of abstract 
         reasoning ability from limited examples in which solutions require some 
         "core knowledge" of concepts such as objects, goal states, counting, and 
         basic geometry. GPT-4 solves only 13/50 of the most straightforward ARC 
         tasks when using textual encodings for their two-dimensional input-output grids. 
         Our failure analysis reveals that GPT-4's capacity to identify objects and 
         reason about them is significantly influenced by the sequential nature of 
         the text that represents an object within a text encoding of a task. 
         To test this hypothesis, we design a new benchmark, the 1D-ARC, which 
         consists of one-dimensional (array-like) tasks that are more conducive 
         to GPT-based reasoning, and where it indeed performs better than on 
         the (2D) ARC. To alleviate this issue, we propose an object-based 
         representation that is obtained through an external tool, resulting in 
         nearly doubling the performance on solved ARC tasks and near-perfect scores 
         on the easier 1D-ARC. Although the state-of-the-art GPT-4 is unable to 
         "reason" perfectly within non-language domains such as the 1D-ARC or a 
         simple ARC subset, our study reveals that the use of object-based representations 
         can significantly improve its reasoning ability. Visualizations, GPT logs, and 
         data are available at this https URL."""
]

In [8]:
print(f"Using Endpoints: {openai.api_base} ...\n")
for user_prompt in user_prompts:
    prompt = f"""Summarize the text delimited by triple backticks 
              ```{user_prompt}``` Use simple, compound, and compound-complex sentences, with 
              no more than one paragraph and no more than five sentences. 
              Keep summary succinct and cohesive, and maintain the original content's tone. """
    response = get_commpletion(client, MODEL, system_content, prompt)
    print(f"\nOriginal content: {user_prompt}")
    print(f"\nSummary  content: {response}")

Using Endpoints: https://console.endpoints.anyscale.com/m/v1 ...


Original content:  The emergence of large language models (LLMs) has marked a significant 
         breakthrough in natural language processing (NLP), leading to remarkable 
         advancements in text understanding and generation. 
         
         Nevertheless, alongside these strides, LLMs exhibit a critical tendency 
         to produce hallucinations, resulting in content that is inconsistent with 
         real-world facts or user inputs. This phenomenon poses substantial challenges 
         to their practical deployment and raises concerns over the reliability of LLMs 
         in real-world scenarios, which attracts increasing attention to detect and 
         mitigate these hallucinations. In this survey, we aim to provide a thorough and 
         in-depth  overview of recent advances in the field of LLM hallucinations. 
         
         We begin with an innovative taxonomy of LLM hallucinations, then de

## Text or information extraction

Another natural langauge capability, similar to summarization or text completion, is extracting key idea or infromation from an article, blog, or a paragraph. For example,
given a set of text, you can ask LLM to extract key ideas or topics or subjects. Or even
better enumerate key takeways for you, saving time if you are in a hurry.

Let's see *how-to* do it by first looking at a simple example, and then progressing into a more complex one.

### Task 1: 
 * summarize the product review
 * extract any information about shipping and packaging for shipping department
 * classify the sentiment of the review: positive or negative.
 * use precise, specific prompt to acheive the task

In [9]:
system_content = "You are master of all knowledge about history, literature, science, social science, philosophy, religion, economics, sports, etc."
product_review = """I got this Australian Bush Baby with soft fur for my niece's birthday, and she absolutely loves it, carrying it around everywhere. The fur is exceptionally soft, and its adorable face gives off a friendly vibe. While I find it a bit smaller than anticipated for the price, my niece's joy makes it worthwhile. What pleasantly surprised me was the early arrival; it came a day earlier than expected. I appreciated the prompt delivery, and the packaging was secure, ensuring the Bush Baby with soft fur arrived in perfect condition. This allowed me to play with it myself before presenting it to my niece."""     

In [10]:
prompt = f"""
Your task is to generate a short summary of a product 
review from an Australian e-commerce site to offer feedback to the 
shipping deparmtment. 

First, provide a short summary the review below, delimited by triple 
backticks, in two sentences: a simple and compound sentence. Second focus 
on any aspects packaging or shipping of the product, and label it as 
"Shipping Department:".  Third, indicate if the review is positive or negative, 
and label it as "Sentiment:"
Review: ```{product_review}``` """

In [11]:
print(f"Using Endpoints: {openai.api_base} ...\n")
response = get_commpletion(client, MODEL, system_content, prompt)
print(f"""\nSummary: {response.replace("```", "")}""")

Using Endpoints: https://console.endpoints.anyscale.com/m/v1 ...


Summary:   Summary: A customer purchased an Australian Bush Baby plush toy with soft fur for their niece's birthday and was pleased with the product's quality and prompt delivery.

Shipping Department: The reviewer appreciated the early arrival of the product, which was delivered a day earlier than expected. The packaging was also secure, ensuring the product arrived in perfect condition.

Sentiment: Positive


### Task 2
 * Given a passage from an article, extract the main theme of the passage and label it as the `Subjects`, if more than one, separated by comma.
 * Identify three key takeways and enumerate them in simple sentences

In [12]:
system_content = "You are master of all knowledge about history, literature, science, social science, philosophy, religion, economics, sports, etc."
            
user_prompts = ["Isaac Newton sat under a tree when an apple fell, an event that, \
                according to popular legend, led to his contemplation of the forces\
                of gravity. Although this story is often regarded as apocryphal or at \
                least exaggerated, it serves as a powerful symbol of Newton's insight \
                into the universal law that governs celestial and earthly bodies alike. \
                His formulation of the law of universal gravitation was revolutionary, \
                as it provided a mathematical explanation for both the motion of planets \
                and the phenomena observed on Earth. Newton's work in physics, captured \
                in his seminal work Philosophiæ Naturalis Principia Mathematica, laid the \
                groundwork for classical mechanics. His influence extended beyond his own \
                time, shaping the course of scientific inquiry for centuries to come."
               ]


In [13]:
print(f"Using Endpoints: {openai.api_base} ...\n")
for text in user_prompts:
    prompt = f""" Given ```{text}``` delimited with triple backticks, identify a single key idea being discussed, 
    and label its 'Subject'. Next, enumerate at most three takeways. 
    Use short, simple sentences. """
    response = get_commpletion(client, MODEL, system_content, prompt)
    print(f"\nOriginal content: {text}")
    print(f"\n {response}")

Using Endpoints: https://console.endpoints.anyscale.com/m/v1 ...


Original content: Isaac Newton sat under a tree when an apple fell, an event that,                 according to popular legend, led to his contemplation of the forces                of gravity. Although this story is often regarded as apocryphal or at                 least exaggerated, it serves as a powerful symbol of Newton's insight                 into the universal law that governs celestial and earthly bodies alike.                 His formulation of the law of universal gravitation was revolutionary,                 as it provided a mathematical explanation for both the motion of planets                 and the phenomena observed on Earth. Newton's work in physics, captured                 in his seminal work Philosophiæ Naturalis Principia Mathematica, laid the                 groundwork for classical mechanics. His influence extended beyond his own                 time, shaping the course of scientific inquiry 

Let's try another example to extract more than on subject or topic being
discussed in the text, and enumerate three takeways.

(Incidentally, I'm reading biography of Benjamin Franklin by Issac Stevenson, and all this seems to align with his career path and passion.)

In [14]:
user_stories = [""""Printer.
                He that has a Trade has an Office of Profit and Honour’ Poor Richard’s Almanack
Benjamin Franklin had an affinity with print and books throughout his life. 
Apprenticed as a child to his brother James, a printer, he mastered all aspects of
the trade, from typesetting to engraving, learning the latest techniques during his
first visit to London.  An avid reader, Franklin saved money to buy books by 
temporarily turning vegetarian and, once settled in Philadelphia, founded the 
Library Company, the first subscription library in the colonies.  As an elder
statesman, he even bought type and kept a press during his stay in France. 
After working as a printer’s journeyman, he set up his own Philadelphian printing 
office in 1728.  His success with the Pennslyannia Gazette and Poor Richard’s
Almanack helped to provide Franklin with the financial means to retire from
business, retaining a stake in his print shop and founding others throughout the 
colonies.  Print also gave him a public voice: Franklin preferred the printed word, 
rather than public rhetoric, influencing political and public opinion as a brilliant
journalist and pamphleteer.

Silence Dogood and the New­England Courant
    When James Franklin lost the contract to print the Boston Gazette, he determined
to begin his own newspaper, launching the New­England Courant in 1721.
Benjamin, who had been indentured secretly to James, helped to print the weekly 
paper.  One night he slipped a composition under the door, beginning the series
of ‘Silence Dogood’ letters, the purported epistles of a vocal widower, with strong 
opinions on drunks, clergymen, foolish fashions and Boston nightlife. Owing no
little debt to the satire of the London Spectator, the letters represented a 
remarkable literary achievement for the 16­year old.  The British Library’s copy has 
been uniquely annotated in what appears to be Franklin’s hand. The first 
‘Dogood’ letter appears on the bottom right.

‘The Main Design of the Weekly Paper will be to Entertain the Town’
    Benjamin’s brother, James, began the New­England Courant in the face of
opposition from the Boston Establishment.  He soon irritated them with his squibs
and satires on the great and the good, attacking the influential clergyman Cotton
Mather’s pet project of small pox inoculation and the authorities’ weak response 
to piracy. Twice arrested, James temporally left the paper in Benjamin’s hands, and 
then continued to publish it under Benjamin’s name to escape a ban on
publication.  This issue is the first printed item to carry the imprint ‘B. Franklin’ (on
the rear).  Franklin announces his intention to ‘Entertain the Town’ on this page.
"""]

In [15]:
print(f"Using Endpoints: {openai.api_base} ...\n")
for story in user_stories:
    prompt = f""" Extract five subjects that are being discussed in the 
                  following text, which is delimited by triple backticks.
                  Format your response as a list of subjects "Subjects:" separated by commas.
                  Make each subject at most two words long, not longer. 
                  Next, enumerate  as a list three takeways, and label them as "Takeways:" 
                  Use short, simple sentences for your takeways.
                  Text sample: '''{story}'''
                  """
    response = get_commpletion(client, MODEL, system_content, prompt)
    print(f"\nComplete Storey: {story}")
    print(f"\n {response}")

Using Endpoints: https://console.endpoints.anyscale.com/m/v1 ...


Complete Storey: "Printer.
                He that has a Trade has an Office of Profit and Honour’ Poor Richard’s Almanack
Benjamin Franklin had an affinity with print and books throughout his life. 
Apprenticed as a child to his brother James, a printer, he mastered all aspects of
the trade, from typesetting to engraving, learning the latest techniques during his
first visit to London.  An avid reader, Franklin saved money to buy books by 
temporarily turning vegetarian and, once settled in Philadelphia, founded the 
Library Company, the first subscription library in the colonies.  As an elder
statesman, he even bought type and kept a press during his stay in France. 
After working as a printer’s journeyman, he set up his own Philadelphian printing 
office in 1728.  His success with the Pennslyannia Gazette and Poor Richard’s
Almanack helped to provide Franklin with the financial means to retire from
business, retainin

## Text classification or sentiment analysis

Unlike classical or traditional machine learning, where you'll have to do supervised learning to collect data, label it, and train for hours, depending on how much data,classifying text using LLM is simple.

In short, you'll have to build a ML model to understand text and classify its sentiments as positive, negative or neutral. 

This onus task is easily done with LLM via clever prompting. 

Let's see what I mean in this *how-to* idenfity sentiments in text But first let's 
generatre some sentiments as our ground truth, and supply them to LLM to observe it
LLM identifies them correctly. This bit is not needed, for I'm just curious.

Positive: "This movie is a true cinematic gem, blending an engaging plot with superb performances and stunning visuals. A masterpiece that leaves a lasting impression."

*Negative*: "Regrettably, the film failed to live up to expectations, with a convoluted storyline, lackluster acting, and uninspiring cinematography. A disappointment overall."

*Neutral*: "The movie had its moments, offering a decent storyline and average performances. While not groundbreaking, it provided an enjoyable viewing experience."

*Positive*: "This city is a vibrant tapestry of culture, with friendly locals, historic landmarks, and a lively atmosphere. An ideal destination for cultural exploration."

*Negative*: "The city's charm is overshadowed by traffic congestion, high pollution levels, and a lack of cleanliness. Not recommended for a peaceful retreat."

*Neutral*: "The city offers a mix of experiences, from bustling markets to serene parks. An interesting but not extraordinary destination for exploration."

*Positive*: "This song is a musical masterpiece, enchanting listeners with its soulful lyrics, mesmerizing melody, and exceptional vocals. A timeless classic."

*Negative*: "The song fails to impress, featuring uninspiring lyrics, a forgettable melody, and lackluster vocals. It lacks the creativity to leave a lasting impact."

*Neutral*: "The song is decent, with a catchy tune and average lyrics. While enjoyable, it doesn't stand out in the vast landscape of music."

*Positive*: "A delightful cinematic experience that seamlessly weaves together a compelling narrative, strong character development, and breathtaking visuals."

*Negative*: "This film, unfortunately, falls short with a disjointed plot, subpar performances, and a lack of coherence. A disappointing viewing experience."

*Neutral*: "While not groundbreaking, the movie offers a decent storyline and competent performances, providing an overall satisfactory viewing experience."

*Positive*: "This city is a haven for culture enthusiasts, boasting historical landmarks, a rich culinary scene, and a welcoming community. A must-visit destination."

*Negative*: "The city's appeal is tarnished by overcrowded streets, noise pollution, and a lack of urban planning. Not recommended for a tranquil getaway."

*Neutral*: "The city offers a diverse range of experiences, from bustling markets to serene parks. An intriguing destination for those seeking a mix of urban and natural landscapes."

In [16]:
system_content = """You are a prominent critic of landscapes, architecture, cities, movies, songs, 
                    entertainment, and a cultural ombudsman. """

user_sentiments = [ "This movie is a true cinematic gem, blending an engaging plot with superb performances and stunning visuals. A masterpiece that leaves a lasting impression.",
                    "Regrettably, the film failed to live up to expectations, with a convoluted storyline, lackluster acting, and uninspiring cinematography. A disappointment overall.",
                    "The movie had its moments, offering a decent storyline and average performances. While not groundbreaking, it provided an enjoyable viewing experience.",
                    "This city is a vibrant tapestry of culture, with friendly locals, historic landmarks, and a lively atmosphere. An ideal destination for cultural exploration.",
                    "The city's charm is overshadowed by traffic congestion, high pollution levels, and a lack of cleanliness. Not recommended for a peaceful retreat.",
                    "The city offers a mix of experiences, from bustling markets to serene parks. An interesting but not extraordinary destination for exploration.",
                    "This song is a musical masterpiece, enchanting listeners with its soulful lyrics, mesmerizing melody, and exceptional vocals. A timeless classic.",
                    "The song fails to impress, featuring uninspiring lyrics, a forgettable melody, and lackluster vocals. It lacks the creativity to leave a lasting impact.",
                    "The song is decent, with a catchy tune and average lyrics. While enjoyable, it doesn't stand out in the vast landscape of music.",
                    "A delightful cinematic experience that seamlessly weaves together a compelling narrative, strong character development, and breathtaking visuals.",
                    "This film, unfortunately, falls short with a disjointed plot, subpar performances, and a lack of coherence. A disappointing viewing experience.",
                    "While not groundbreaking, the movie offers a decent storyline and competent performances, providing an overall satisfactory viewing experience.",
                    "This city is a haven for culture enthusiasts, boasting historical landmarks, a rich culinary scene, and a welcoming community. A must-visit destination.",
                    "The city's appeal is tarnished by overcrowded streets, noise pollution, and a lack of urban planning. Not recommended for a tranquil getaway.",
                    "The city offers a diverse range of experiences, from bustling markets to serene parks. An intriguing destination for those seeking a mix of urban and natural landscapes."
]

In [17]:
print(f"Using Endpoints: {openai.api_base} ...\n")
for user_sentiment in user_sentiments:
    prompt = f"""What is the sentiment of the ```{user_sentiment}`` which is delimited with triple backticks?
                  Classify the given text into single label as neutral, negative 
                  or positive. Do not expand on your response. Use a single word.
                  If you cannot classify do not guess, just label as dont' know.
                    """
    response = get_commpletion(client, MODEL, system_content, prompt)
    print(f"\nSentiment: {user_sentiment}")
    print(f"\nLabel    : {response}")

Using Endpoints: https://console.endpoints.anyscale.com/m/v1 ...


Sentiment: This movie is a true cinematic gem, blending an engaging plot with superb performances and stunning visuals. A masterpiece that leaves a lasting impression.

Label    :   Positive

Sentiment: Regrettably, the film failed to live up to expectations, with a convoluted storyline, lackluster acting, and uninspiring cinematography. A disappointment overall.

Label    :   Negative

Sentiment: The movie had its moments, offering a decent storyline and average performances. While not groundbreaking, it provided an enjoyable viewing experience.

Label    :   Neutral

Sentiment: This city is a vibrant tapestry of culture, with friendly locals, historic landmarks, and a lively atmosphere. An ideal destination for cultural exploration.

Label    :   Positive

Sentiment: The city's charm is overshadowed by traffic congestion, high pollution levels, and a lack of cleanliness. Not recommended for a peaceful retreat.

Label 

## Text categorization
Like sentiment analysis, given a query, an LLM can identify from its context how to classify and route customer queries to respective departments. Also, note that LLM can detect foul language and respond politely. Text categorization can be employed to automate customer on-line queries.

Let's look at how we can achieve that with smart and deliberate prompting.

<img src="./images/category_resp.png" height="35%" width="%65">



In [18]:
system_content = """You are a smart and helful Assistant who can route customer queries to 
                    respective customer service departments.
                    """

customer_queries = ["""My modem has stop working. I tried to restart but the orange light keep flashing. It never turns green.""",
                    """I just moved into town, and I need Internet service""",
                    """Why am I being charged extra $20 a month for cable TV when I don't use a television?""",
                    """I need to change my user name and password since someone is using my credentials""",
                    """What days this week are we having a general upgrades to the cable models?""",
                    """What day is the best day to call customer service so that I can avoid talking to a bloody bot!""",
                    """Your company is full of incompetent fools!""",
                    """I hate your worthless services. Cancel my stupid account or else I'll sue you!"""
                   ]
                    

In [19]:
print(f"Using Endpoints: {openai.api_base} ...\n")
for query in customer_queries:
    prompt = f""" Classify each customer {query} into the following categories:
                    1. Technical support
                    2. Billing 
                    3. Account Management
                    4. New Customer  
                    5. General inquiry
                  Do not expand your response. Only use the above categories. 
                  If you can't precisely categorize, then default to "General inquiry."
                  If customer {query} is abusive or in a foul language, then respond with 
                  "No need for foul language. Please be respectful."
    """
    response = get_commpletion(client, MODEL, system_content, prompt)
    print(f"\nQuery: {query}")
    print(f"Route to: {response}\n")

Using Endpoints: https://console.endpoints.anyscale.com/m/v1 ...


Query: My modem has stop working. I tried to restart but the orange light keep flashing. It never turns green.
Route to:   Technical support


Query: I just moved into town, and I need Internet service
Route to:   New Customer


Query: Why am I being charged extra $20 a month for cable TV when I don't use a television?
Route to:   General inquiry


Query: I need to change my user name and password since someone is using my credentials
Route to:   General inquiry


Query: What days this week are we having a general upgrades to the cable models?
Route to:   General inquiry


Query: What day is the best day to call customer service so that I can avoid talking to a bloody bot!
Route to:   General inquiry


Query: Your company is full of incompetent fools!
Route to:   General inquiry.

No need for foul language. Please be respectful.


Query: I hate your worthless services. Cancel my stupid account or else I'll sue you!
Rout

## Text transation and transformation

Language translation by far is the most common use case for natural language processing. 
We have seen its early uses in Google translation, but with the emergence of multi-lingual LLMs, this task is simply achieved by exact prompting. 

In this section, we'll explore tasks in how to use LLMs for text translations, langugage identication, text transformation, spelling and grammar checking, tone adjustment, and format conversion.

### Task 1:
 * Given an English text, translate into French, Spanish, and German.
 * Given a foreign language text, idenfify the language, and translate to English.


👷‍♀️ under construction 🚧

In [20]:
system_content= """You are a world reknowned supreme lingiust and a universal translator. You are a polglot, and fluently speak many global languages"""

english_texts = [""" Welcome to New York for the United Nations General Council Meeting. Today
is a special day for us to celeberate all our achievments since this global institute's formation.
But more importantly, we want to address how we can mitigate global conflict with conversation
and promote deterence, detente, and discussion."""
]

In [21]:
print(f"Using Endpoints: {openai.api_base} ...\n")
for english_text in english_texts:
    prompt = f""""Given an English text in triple ticks '''{english_text}'''. Translate into
three languases: Spanish, French, German, and Mandarin. 
Label each translation with the langauge Name: followed by translation on a seperate line."""
    response = get_commpletion(client, MODEL, system_content, prompt)
    print(f"\nEnglish Text: {english_text}")
    print(f"{response}\n")
                

Using Endpoints: https://console.endpoints.anyscale.com/m/v1 ...


English Text:  Welcome to New York for the United Nations General Council Meeting. Today
is a special day for us to celeberate all our achievments since this global institute's formation.
But more importantly, we want to address how we can mitigate global conflict with conversation
and promote deterence, detente, and discussion.
  Sure, here are the translations of the given English text into three languages:

Name: Spanish
Translation:
Bienvenidos a Nueva York para la Reunión del Consejo General de las Naciones Unidas. Hoy es un día especial para celebrar todos nuestros logros desde la formación de esta institución global. Pero lo más importante es abordar cómo podemos mitigar los conflictos globales a través de la conversación y fomentar la détente, el diálogo y la discusión.

Name: French
Translation:
Bienvenue à New York pour la Réunion du Conseil Général des Nations Unies. Aujourd'hui est un jour spécial pour céléb

Given a foreing language, identify the language and translate into English.

This is the reverse of the above.

In [22]:
languages_texts = ["""Bienvenidos a Nueva York para la Reunión del Consejo General de las Naciones Unidas. Hoy
es un día especial para celebrar todos nuestros logros desde la formación de este instituto global.
Pero más importante aún, queremos abordar cómo podemos mitigar el conflicto global con conversaciones
y promover la disuasión, la distensión y el diálogo.""",
            """Willkommen in New York zur Sitzung des Allgemeinen Rates der Vereinten Nationen. Heute
ist ein besonderer Tag für uns, um all unsere Errungenschaften seit der Gründung dieses globalen Instituts zu feiern.
Aber wichtiger ist, dass wir ansprechen möchten, wie wir globale Konflikte durch Gespräche mildern können
und Abschreckung, Entspannung und Diskussion fördern.""",
                  """Bienvenue à New York pour la réunion du Conseil Général des Nations Unies. Aujourd'hui,
c'est un jour spécial pour nous pour célébrer toutes nos réalisations depuis la formation de cette institution mondiale.
Mais plus important encore, nous voulons aborder comment nous pouvons atténuer les conflits mondiaux grâce à la conversation
et promouvoir la dissuasion, la détente et la discussion.""",
                  """欢迎来到纽约参加联合国大会议。今天对我们来说是一个特别的日子，我们将庆祝自该全球机构成立以来取得的所有成就。但更重要的是，我们想要讨论如何通过对话来缓解全球冲突，并促进遏制、缓和和讨论。
"""]


In [23]:
print(f"Using Endpoints: {openai.api_base} ...\n")
for language_text in languages_texts:
    prompt = f""""Given a language text in triple ticks '''{language_text}'''. Idenfity
    the language with the langauge Name: followed by an English translation on a seperate line, labeled as English translation:"""
    response = get_commpletion(client, MODEL, system_content, prompt)
    print(f"\nLanguage Text: {language_text}")
    print(f"{response}\n")
                

Using Endpoints: https://console.endpoints.anyscale.com/m/v1 ...


Language Text: Bienvenidos a Nueva York para la Reunión del Consejo General de las Naciones Unidas. Hoy
es un día especial para celebrar todos nuestros logros desde la formación de este instituto global.
Pero más importante aún, queremos abordar cómo podemos mitigar el conflicto global con conversaciones
y promover la disuasión, la distensión y el diálogo.
  Sure, I can help you with that!

Language: Spanish

English translation:
Welcome to New York for the United Nations General Assembly meeting. Today is a special day to celebrate all our achievements since the establishment of this global institution. But more importantly, we want to discuss how we can mitigate global conflict through conversations and promote dissuasion, detente, and dialogue.


Language Text: Willkommen in New York zur Sitzung des Allgemeinen Rates der Vereinten Nationen. Heute
ist ein besonderer Tag für uns, um all unsere Errungenschaften seit der

### Task 2

 * Given an English text, proof read it and correct any grammatical and usage errors.
 * Given a Pirate text, correct its tone to standard English.


In [24]:
system_content = """You are a fastidious grammarian. You can proofread any English text and convert to 
its grammtical correct and usage form."""

bad_english_texts = ["""I don't know nothing about them big words and grammar rules. Me and my friend, we was talking, and he don't agree with me. We ain't never gonna figure it out, I reckon. His dog don't listen good, always running around and don't come when you call.""",
                     """Yesterday, we was at the park, and them kids was playing. She don't like the way how they acted, but I don't got no problem with it. We seen a movie last night, and it was good, but my sister, she don't seen it yet. Them books on the shelf, they ain't interesting to me."""
                    ]

In [25]:
print(f"Using Endpoints: {openai.api_base} ...\n")
for bad_english_text in bad_english_texts:
    prompt = f""""Proofread and correct the text provided in triple ticks '''{bad_english_text}'''.
    Use standard usage and remedy any incorect grammar usage.
    """
    response = get_commpletion(client, MODEL, system_content, prompt)
    print(f"\nOriginal Text: {bad_english_text}")
    print(f"Corrected  Text: {response}\n")

Using Endpoints: https://console.endpoints.anyscale.com/m/v1 ...


Original Text: I don't know nothing about them big words and grammar rules. Me and my friend, we was talking, and he don't agree with me. We ain't never gonna figure it out, I reckon. His dog don't listen good, always running around and don't come when you call.
Corrected  Text:   Here's the corrected text:

"I don't know anything about those big words and grammar rules. My friend and I were talking, and he didn't agree with me. We'll never figure it out, I reckon. His dog doesn't listen well, always running around and not coming when called."

Corrected usage:

* "I don't know nothing" should be "I don't know anything."
* "Me and my friend" should be "My friend and I."
* "We was talking" should be "We were talking."
* "He don't agree" should be "He didn't agree."
* "We ain't" should be "We'll never."
* "His dog don't" should be "His dog doesn't."
* "Don't come" should be "Doesn't come."


Original Text: Yesterday, we w

In [26]:
pirate_texts = ["""Arrr matey! I be knowin' nuthin' 'bout them fancy words and grammatical rules. Me and me heartie, we be chattin', and he don't be agreein' with me. We ain't never gonna figure it out, I reckon. His scallywag of a dog don't be listenin' well, always runnin' around and not comin' when ye call."""
                       ]

In [27]:
print(f"Using Endpoints: {openai.api_base} ...\n")
for pirate_text in pirate_texts:
    prompt = f""""Convert the Pirate text provided in triple ticks '''{pirate_text}'''.
    Use standard usage and remedy any incorect grammar usage, dropping all Pirate greetings.
    """
    response = get_commpletion(client, MODEL, system_content, prompt)
    print(f"\nOriginal Text: {pirate_text}")
    print(f"\nCorrected  Text: {response}\n")

Using Endpoints: https://console.endpoints.anyscale.com/m/v1 ...


Original Text: Arrr matey! I be knowin' nuthin' 'bout them fancy words and grammatical rules. Me and me heartie, we be chattin', and he don't be agreein' with me. We ain't never gonna figure it out, I reckon. His scallywag of a dog don't be listenin' well, always runnin' around and not comin' when ye call.

Corrected  Text:   Here's the corrected text:

"I don't know anything about those fancy words and grammatical rules. My friend and I were chatting, and he didn't agree with me. We'll never figure it out, I reckon. His mischievous dog doesn't listen well, always running around and not coming when called."

I made the following corrections:

* Changed "Arrr matey!" to "I don't know anything" to drop the Pirate greeting.
* Changed "be knowin'" to "know" to correct the verb tense.
* Changed "ain't" to "don't" to correct the negative form of the verb "to be."
* Changed "reckon" to "reckon" to keep the word, but adjust the

### Task 3
* Given some text in a particular format, convert it into JSON format.
* For example, we LLM to producce names of three top shoes, but we want them it product and its items in JSON format. This JSON format can be fed downstream into another application that may process it.

Let's have go at it.


In [28]:
system_content = """You have knowledge of all sporting goods and will provide knowledge answers
to queries about sporting goods."""

In [31]:
print(f"Using Endpoints: {openai.api_base} ...\n")
prompt = f"""Generate five distinct products on training shoes. Generate products and format them all as a 
            in a single JSON object. For each product, the JSON object should contain items: Brand, Description, Size, Gender: Male 
            or Female or Unisex, Price, and at least three customer reviews as Review 
            item"""
response = get_commpletion(client, MODEL, system_content, prompt)
print(f"{response}\n")

Using Endpoints: https://console.endpoints.anyscale.com/m/v1 ...

  Here are five distinct products on training shoes, formatted as a JSON object with the specified items:

{
"Product 1": {
"Brand": "Nike",
"Description": "The Nike Air Zoom Pegasus 37 is a versatile training shoe that's great for runners who need a little extra support. It features a responsive Zoom Air unit and a supportive, stability-enhancing design.",
"Size": "8-14",
"Gender": "Male",
"Price": "100.00",
"Reviews": [
{
"Name": "John Doe",
"Rating": 4.5,
"Comment": "I've been running in these shoes for a few weeks now and I have to say, they're fantastic. The support and cushioning are top-notch and they feel great on my feet."
},
{
"Name": "Jane Smith",
"Rating": 4.0,
"Comment": "I was a bit skeptical about the stability feature at first, but I have to say, it really does help. I no longer feel like I'm going to roll my ankle when I run. Definitely recommend!"
},
{
"Name": "Bob Johnson",
"Rating": 4.2,
"Comment": "I

## Simple and complex reasoning 

An import characteristic of LLM is that it's not only general respository of compressed
knowledge garned from large corpus of text, but can be employed as a simple and complex reasoning engine. With use of precise prompt, you can instruct LLM to think trough a problem in a step by step fashion.

Let's look at some tasks as examples.
 * **Task 1**: given a list of numbers identify the prime numbers, add the prime numbers and check if the sum is even or odd.
 * **Task 2**: given an hourly rate of wages, compute your yearly income if you work 30 hours a week

In [39]:
system_content = """You are a reasoning engine. Given a problem think through the problem logically
in a step by step manner."""

In [40]:
# Task 1 prompt
prime_number_prompt = f"""given a list of numbers 1,2,3,4,5,7,8, 11,13,17,19,23,24,29 identify the prime numbers, add the prime numbers, 
and check if the sum is even or odd. Explain each step how you solved the problem"""

In [42]:
#Task 2 prompt
hourly_wages_prompt = f"""If my hourly rate is $117.79 per hour and 30 hours a week, what
is my yearly income? Break the problem into simple steps and explain in each step how you arrive 
to the answer. If you don't know, simple say I don't know. Do not make up answers"""


#### Task 1

In [44]:
response = get_commpletion(client, MODEL, system_content, prime_number_prompt)
print(f"{response}\n")

  Sure! Here's how I would approach the problem:

Step 1: Identify the prime numbers in the list.
To do this, I would use a definition of a prime number: a prime number is a positive integer that is divisible only by itself and 1.
I would then scan the list and identify the numbers that meet this definition: 11, 13, 17, 19, 23, and 29.

Step 2: Add the prime numbers together.
To do this, I would simply add the identified prime numbers together:
11 + 13 + 17 + 19 + 23 + 29 = 113

Step 3: Check if the sum is even or odd.
To do this, I would use the definition of an even number: an even number is a number that is divisible by 2.
I would then check if 113 is divisible by 2: it is not, since 113 / 2 = 56.5 (not an integer).
Therefore, the sum of the prime numbers in the list is odd.

So, to summarize:

* The prime numbers in the list are 11, 13, 17, 19, 23, and 29.
* The sum of these prime numbers is 113.
* The sum is odd.

I hope this explanation helps! Let me know if you have any further 

#### Task 2

In [45]:
response = get_commpletion(client, MODEL, system_content, hourly_wages_prompt)
print(f"{response}\n")

  Sure, I can help you with that! Here's how I would approach the problem:

Step 1: Convert the hourly rate to a monthly rate

First, let's convert the hourly rate to a monthly rate. There are approximately 4 weeks in a month, so we can multiply the hourly rate by 4 to get the monthly rate:

$117.79/hour x 4 weeks/month = $471.16/month

Step 2: Calculate the monthly income for 30 hours/week

Next, we need to calculate the monthly income for 30 hours/week. We can multiply the monthly rate by the number of hours per week:

$471.16/month x 30 hours/week = $14,134.80/month

Step 3: Calculate the yearly income

Finally, we can calculate the yearly income by multiplying the monthly income by 12:

$14,134.80/month x 12 months/year = $169,617.60/year

Therefore, if your hourly rate is $117.79 per hour and you work 30 hours a week, your yearly income would be approximately $169,617.60.

I hope that helps! Let me know if you have any questions or if you'd like me to clarify any of the steps.



## Code generation

Language models like ChatGPT and Llama 2 are really good at generating code. Copilot on GitHub is a cool example of this. You can do lots of different code tasks just by asking in a smart way. Let's check out a few examples to see how it's helpful.

#### Task 1
 * Generate Python code to compute the value of PI using Ray distributed framework



In [46]:
system_content = """You are a supreme CoPilot for developer. Given a task you can
generate code for that task."""

In [47]:
python_code_prompt="""Generate Python code to compute the value of PI using Ray 
distributed framework API. Use the Monte Carlo method to compute the value of PI.
Include in-line comments explaining the code"""

In [48]:
response = get_commpletion(client, MODEL, system_content, python_code_prompt)
print(f"{response}\n")

  Sure! Here's an example Python code that uses the Ray distributed framework API to compute the value of PI using the Monte Carlo method:
```python
import ray
import numpy as np

# Define the number of samples to use for the Monte Carlo estimation
num_samples = 10000

# Define the number of processors to use for the computation
num_processors = 4

# Initialize Ray
ray.init(num_processors)

# Define a function to compute the Monte Carlo estimate of PI
@ray.task
def monte_carlo_pi(samples):
    # Generate random points inside a square
    points = np.random.rand(samples, 2)

    # Compute the distance from the origin for each point
    distances = np.linalg.norm(points, axis=1)

    # Count the number of points inside the quarter-circle
    inside_circle = np.count_elements(distances < 1)

    # Compute the estimate of PI using the Monte Carlo method
    pi_estimate = 4 * inside_circle / samples

    return pi_estimate

# Compute the value of PI using the Monte Carlo method
result = ray

#### Task 2
 * Given SQL schema tables, generate an SQL query 



In [49]:
sql_code_prompt="""Given the following SQL schema for tables
Table clicks, columns = [target_url, orig_url, user_id, clicks]
Table users, columns = [user_id, f_name, l_name, e_mail, company, title], generate
an SQL query that computes in the descening order of all the clicks. Also, for
each user_id, list the f_name, l_name, company, and title
"""

In [50]:
response = get_commpletion(client, MODEL, system_content, sql_code_prompt)
print(f"{response}\n")

  Sure! Here's an SQL query that should accomplish the task you described:
```sql
SELECT clicks.user_id, users.f_name, users.l_name, users.company, users.title, clicks.clicks
FROM clicks
INNER JOIN users ON clicks.user_id = users.user_id
ORDER BY clicks.clicks DESC;
```
This query joins the `clicks` and `users` tables on the `user_id` column, and then sorts the results by the `clicks` column in descending order. The `SELECT` clause retrieves the `user_id`, `f_name`, `l_name`, `company`, and `title` columns from the `users` table, as well as the `clicks` column from the `clicks` table.

Here's a breakdown of the query:

1. `SELECT clicks.user_id, users.f_name, users.l_name, users.company, users.title, clicks.clicks`: This clause selects the columns that we want to include in the output. We're selecting the `user_id` column from the `clicks` table, and the `f_name`, `l_name`, `company`, and `title` columns from the `users` table. We're also selecting the `clicks` column from the `clicks`

## All this is amazing! 😜 Feel the wizardy prompt power 🧙‍♀️